# Proyecto Final Topicos de Industria I

**Nombre:** Judith Guadalupe Navarro Flores

**e-mail:** judith.navarro4332@alumnos.udg.mx

**Nombre:** Marco Antonio Hernández Preciado  
**e-mail:** marco.hpreciado@alumnos.udg.mx

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')
pn.extension()

## Carga del dataset

In [ ]:
# Paso 1: Cargar el Dataset
jalisco_pea = pd.read_csv('jalisco_pea.csv')

# Imprimir las primeras filas del dataframe para verificar la carga
print(jalisco_pea.head())

   ENTIDAD  NOM_ENT  MUN NOM_MUN  LOC NOM_LOC AGEB  MZA  POBTOT  PEA
0       14  Jalisco    1  Acatic    1  Acatic  194   18      66   33
1       14  Jalisco    1  Acatic    1  Acatic  194   19      61   32
2       14  Jalisco    1  Acatic    1  Acatic  194    2      55   24
3       14  Jalisco    1  Acatic    1  Acatic  194   20      72   31
4       14  Jalisco    1  Acatic    1  Acatic  194   21      61   33


## Creación de widgets

In [ ]:
# Widgets para establecer límites en 'Población Total (POBTOT)'
min_slider_pob = pnw.IntInput(name='Valor Mínimo POBTOT', start=0, end=1380000, step=10, value=0)
max_slider_pob = pnw.IntInput(name='Valor Máximo POBTOT', start=0, end=1380000, step=10, value=1380000)

# Widgets para establecer límites en 'Población Económicamente Activa (PEA)'
min_slider_pea = pnw.IntInput(name='Valor Mínimo PEA', start=0, end=70000, step=10, value=0)
max_slider_pea = pnw.IntInput(name='Valor Máximo PEA', start=0, end=70000, step=10, value=70000)

# Desplegable para seleccionar el municipio
#estado = pnw.Select(name='Municipios ZMG', value='Guadalajara', options=['Zapopan', 'Tlaquepaque', 'Tlajomulco', 'Guadalajara'])

# Crear el selector usando los valores de la columna 'NOM_MUN'
selector = pnw.Select(name='Municipios ZMG', value='Guadalajara', options=jalisco_pea['NOM_MUN'].unique().tolist())

## Funciones de filtrado de datos

In [ ]:
# Funciones de filtrado
def filtrodatos_pob(df, valor_minimo, valor_maximo):
    return df[(df['POBTOT'] >= valor_minimo) & (df['POBTOT'] <= valor_maximo)]

def filtrodatos_pea(df, valor_minimo, valor_maximo):
    return df[(df['PEA'] >= valor_minimo) & (df['PEA'] <= valor_maximo)]

def filtrodatos_estado(df, selector):
    return df[df['NOM_MUN'] == selector]

In [ ]:
# Función de actualizacion para filtrar los datos
@pn.depends(min_slider_pob, max_slider_pob, min_slider_pea, max_slider_pea, selector)
def actualizar_datos(min_slider_pob, max_slider_pob, min_slider_pea, max_slider_pea, selector):
    df = jalisco_pea.copy()
    df = filtrodatos_pob(df, min_slider_pob, max_slider_pob)
    df = filtrodatos_pea(df, min_slider_pea, max_slider_pea)
    df = filtrodatos_estado(df, selector)
    return df

## Creacion de graficos

In [ ]:
# Crear gráficos
def crear_grafico_dispersion(datos):
    grafico_dispersion = px.scatter(datos, x='POBTOT', y='PEA', trendline='ols')
    return pn.pane.Plotly(grafico_dispersion)

def crear_histograma(datos):
    histograma = go.Histogram(x=datos['POBTOT'])
    layout = go.Layout(title='Distribución de la población total')
    fig = go.Figure(data=histograma, layout=layout)
    return pn.pane.Plotly(fig)

In [ ]:
# Crear Tabs para seleccionar el gráfico
tabs = pn.Tabs(
    ('Gráfico de Dispersión', pn.bind(crear_grafico_dispersion, actualizar_datos)),
    ('Histograma', pn.bind(crear_histograma, actualizar_datos))
)

## Creacion y definicion del Dashboard

In [ ]:
# Definir el layout del dashboard
dashboard = pn.Row(
    pn.Column(selector, min_slider_pob, max_slider_pob, min_slider_pea, max_slider_pea),
    tabs
)

## Mostrar Panel Dashboard

In [ ]:
# Servir el dashboard
dashboard.servable()

Row
    [0] Column
        [0] Select(name='Municipios ZMG', options=['Acatic', 'Atemajac de Br...], value='Guadalajara')
        [1] IntInput(end=1380000, name='Valor Mínimo POBTOT', start=0, step=10)
        [2] IntInput(end=1380000, name='Valor Máximo POBTOT', start=0, step=10, value=1380000)
        [3] IntInput(end=70000, name='Valor Mínimo PEA', start=0, step=10)
        [4] IntInput(end=70000, name='Valor Máximo PEA', start=0, step=10, value=70000)
    [1] Tabs
        [0] ParamFunction(function, _pane=Plotly, defer_load=False, name='Gráfico de Dispersión')
        [1] ParamFunction(function, _pane=Plotly, defer_load=False, name='Histograma')